In [ ]:
from constants import *
import os
from dotenv import load_dotenv
import subprocess
import json
from web3 import Web3
from eth_account import Account
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

In [ ]:
load_dotenv()
mnemonic = os.getenv("MNEMONIC", 'deposit quote question hidden brave raw tiny fragile vehicle foil they fabric')

In [ ]:

def derive_wallets (mnemonic, coin, number):
    
    command = f"./derive -g --mnemonic='{mnemonic}' --coin='{coin}' --numderive='{number}' --format=json"
    process = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = process.communicate()
    process_status = process.wait()
    keys = json.loads(output)
    
    return keys

In [ ]:
def coins ():
    coin_dict = {
    'btc-test': derive_wallets(mnemonic, BTCTEST, 3),
    'eth': derive_wallets(mnemonic, ETH, 3)
    }
    return coin_dict

In [ ]:
#test coin function
coins()

In [ ]:
# test private key extract
coins()[ETH][0]['privkey']

In [ ]:
#test btc key extract
btctest_key = coins()[BTCTEST][0]['privkey']
print(btctest_key)

In [ ]:
#create account object that can be consumed by web3 or bit
def priv_key_to_account(coin, priv_key):

    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    else:
        return 'err: unsupported key'




In [ ]:
#set eth testnet address
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.isConnected()

In [ ]:
#set eth sender account
eth_sender = priv_key_to_account(ETH, coins()[ETH][0]['privkey'])

print(eth_sender)


In [ ]:
#set ETH recipient address
print("Enter receiver address")
eth_rec = input()
print(eth_rec)

In [ ]:
#set btc_testnet sender
btc_sender = priv_key_to_account(BTCTEST, coins()[BTCTEST][0]['privkey'])
print(btc_sender.address)
len(btc_sender.address)


In [ ]:
#set btc_testnet recipient
btc_rec = priv_key_to_account(BTCTEST, coins()[BTCTEST][1]['privkey'])
print(btc_rec.address)

In [ ]:
def create_tx(coin, account, to, amount):
    
    if coin == ETH:
        gasEstimate = w3.eth.estimate_gas({
            'from': account.address, 
            'to': to, 
            'value': w3.toWei(amount,'ether') 
            })
        return {
            'to': to, 
            'from': account.address, 
            'value': w3.toWei(amount,'ether'), 
            'gas': gasEstimate, 
            'gasPrice': w3.eth.gas_price, 
            'nonce':w3.eth.get_transaction_count(account.address),
        }
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account, [(to, amount, BTC)])


In [ ]:
#test eth transaction
create_tx(ETH, eth_sender, eth_rec, 0.3)

In [ ]:
PrivateKeyTestnet.get_unspents(btc_sender)

In [ ]:
PrivateKeyTestnet.get_balance(btc_sender, currency='btc')

In [ ]:
#test btc tx
# create_tx(BTCTEST, btc_sender, btc_rec, .0001)

# doesn't work - this is what is failing:
# PrivateKeyTestnet.prepare_transaction(btc_sender.address, [(btc_rec, .0001, BTC)])
#also tried:
PrivateKeyTestnet.get_unspents(btc_sender)
PrivateKeyTestnet.prepare_transaction(btc_sender, [(btc_rec, .0001, BTC)], unspents=btc_sender.get_unspents())
    # Output:
    # Must have at least 1 unspent

In [ ]:
# create function to sign and send tx's
def send_tx (coin, account, to, amount):
    raw_tx = create_tx(coin, account, to, amount)
    signed_tx = account.sign_transaction(raw_tx)
    
    if coin == ETH:
        return w3.eth.sendRawTransaction(signed_tx.rawTransaction)

    elif coin == BTCTEST:
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

In [ ]:
#test ETH send_tx
send_tx(ETH, eth_sender, eth_rec, 3.33)

In [ ]:
#test BTC send_tx
send_tx(BTCTEST, btc_sender, btc_rec, 0.001)